# 우리나라 인구 소멸 위기 지역 분석
- 인구 소멸 위험 지역 : 65세 이상 노인 인구와 20 ~ 39세 여성 인구를 비교하여,
20 ~ 39세 여성/65세 이상 노인 인구 < 0.5 인 지역 <br>
- '국가통계포털 인구 가구' > 인구부문 > 총조사 인구 총괄 > 총조사인구 > 성 연령 및 세대구성별 인구


In [39]:
import pandas as pd
import numpy as np

In [40]:
pop = pd.read_csv('./source/성별, 연령별 인구 - 시군구(2019).csv', sep = ',', encoding='euc-kr')
pop.head()

,광역시도,시군구,행정구,구분,합계,15세미만,15~19세,20~24세,25~29세,30~34세,...,40~44세,45~49세,50~54세,55~59세,60~64세,65~69세,70~74세,75~79세,80~84세,85세이상
0,서울,소계,NaN,계,9096901,983091,402343,548245,768413,687804,...,677660,782455,731258,728273,664875,465250,370242,280007,163363,96792
1,서울,소계,NaN,남자,4386287,505543,201751,233761,377349,341858,...,335606,385491,354886,349208,313417,218378,170303,126711,66754,30757
2,서울,소계,NaN,여자,4710614,477548,200592,314484,391064,345946,...,342054,396964,376372,379065,351458,246872,199939,153296,96609,66035
3,서울,종로구,NaN,계,136649,12204,5801,9816,11972,9459,...,9034,11784,11462,11781,10125,7057,5960,5277,3341,2065
4,서울,종로구,NaN,남자,65594,6183,2803,4230,6167,4747,...,4341,5709,5511,5773,5021,3369,2659,2290,1387,747


## 서울처럼 시군구만 있고 행정구가 없어 NaN이 되는 경우, 경기도처럼 행정구도 있는 경우

즉 NaN을 같은 열의 바로 앞 인덱스(한칸 위)로 채움 <br>
- df.fillna(method = 'pad', inplace = True)

In [41]:
pop.fillna(method='pad', inplace = True)
pop[pop['광역시도']=='경기'].head(7)

,광역시도,시군구,행정구,구분,합계,15세미만,15~19세,20~24세,25~29세,30~34세,...,40~44세,45~49세,50~54세,55~59세,60~64세,65~69세,70~74세,75~79세,80~84세,85세이상
249,경기,소계,NaN,계,12437493,1780295,618620,670267,853024,809456,...,1027268,1161425,1087603,1024664,819471,511765,394666,310540,200400,121659
250,경기,소계,NaN,남자,6186317,912898,314381,312258,445138,414592,...,518975,585812,542592,517387,408870,247975,184317,134215,77861,37784
251,경기,소계,NaN,여자,6251176,867397,304239,358009,407886,394864,...,508293,575613,545011,507277,410601,263790,210349,176325,122539,83875
252,경기,수원시,소계,계,1135358,158249,59415,70240,93957,82625,...,94055,104614,99866,90931,66430,40277,30043,22987,14744,9389
253,경기,수원시,소계,남자,565728,81490,30411,32716,49813,43088,...,47371,52198,48749,45974,33130,19354,13960,9697,5490,2743
254,경기,수원시,소계,여자,569630,76759,29004,37524,44144,39537,...,46684,52416,51117,44957,33300,20923,16083,13290,9254,6646
255,경기,수원시,장안구,계,262957,33148,14970,17775,20325,14990,...,20450,25477,25346,22674,16863,10534,8217,6633,4422,2768


In [42]:
pop.rename(columns={
    '시군구':'시도',
    '합계':'인구수'
},inplace=True)
pop.head()

,광역시도,시도,행정구,구분,인구수,15세미만,15~19세,20~24세,25~29세,30~34세,...,40~44세,45~49세,50~54세,55~59세,60~64세,65~69세,70~74세,75~79세,80~84세,85세이상
0,서울,소계,NaN,계,9096901,983091,402343,548245,768413,687804,...,677660,782455,731258,728273,664875,465250,370242,280007,163363,96792
1,서울,소계,NaN,남자,4386287,505543,201751,233761,377349,341858,...,335606,385491,354886,349208,313417,218378,170303,126711,66754,30757
2,서울,소계,NaN,여자,4710614,477548,200592,314484,391064,345946,...,342054,396964,376372,379065,351458,246872,199939,153296,96609,66035
3,서울,종로구,NaN,계,136649,12204,5801,9816,11972,9459,...,9034,11784,11462,11781,10125,7057,5960,5277,3341,2065
4,서울,종로구,NaN,남자,65594,6183,2803,4230,6167,4747,...,4341,5709,5511,5773,5021,3369,2659,2290,1387,747


## 시도가 소계(=합계)라고 나온 행은 전국, 시 전체 인구를 합친 값. 성별과 연령을 쪼갠 데이터가 아니라 쓸모 없음. 
- 밑에 어짜피 시도로 구분되어 나와있음 <br>
특정 행 제거

In [43]:
# '시도' 열이 '소계'가 아닌 행만 남김, 다시 말해 '소계' 인 행만 제거
pop = pop[pop['시도'] != '소계']
pop.head()

,광역시도,시도,행정구,구분,인구수,15세미만,15~19세,20~24세,25~29세,30~34세,...,40~44세,45~49세,50~54세,55~59세,60~64세,65~69세,70~74세,75~79세,80~84세,85세이상
3,서울,종로구,NaN,계,136649,12204,5801,9816,11972,9459,...,9034,11784,11462,11781,10125,7057,5960,5277,3341,2065
4,서울,종로구,NaN,남자,65594,6183,2803,4230,6167,4747,...,4341,5709,5511,5773,5021,3369,2659,2290,1387,747
5,서울,종로구,NaN,여자,71055,6021,2998,5586,5805,4712,...,4693,6075,5951,6008,5104,3688,3301,2987,1954,1318
6,서울,중구,NaN,계,115665,9839,3896,7511,10304,9636,...,7836,8903,9313,9786,9002,6211,5077,4201,2581,1695
7,서울,중구,NaN,남자,55820,4988,1896,3103,5115,4790,...,4054,4423,4587,4830,4430,2998,2295,1795,1003,523


# 기준에 따라 20 ~ 39, 65세 이상으로 표 정리하기

In [44]:
pop['20~39세'] = pop['20~24세'] + pop['25~29세'] + pop['30~34세'] + pop['35~39세']
pop['65세 이상'] = pop['65~69세'] + pop['70~74세'] + pop['75~79세'] + pop['80~84세'] + pop['85세이상']
pop.head()

,광역시도,시도,행정구,구분,인구수,15세미만,15~19세,20~24세,25~29세,30~34세,...,50~54세,55~59세,60~64세,65~69세,70~74세,75~79세,80~84세,85세이상,20~39세,65세 이상
3,서울,종로구,NaN,계,136649,12204,5801,9816,11972,9459,...,11462,11781,10125,7057,5960,5277,3341,2065,40758,23700
4,서울,종로구,NaN,남자,65594,6183,2803,4230,6167,4747,...,5511,5773,5021,3369,2659,2290,1387,747,19801,10452
5,서울,종로구,NaN,여자,71055,6021,2998,5586,5805,4712,...,5951,6008,5104,3688,3301,2987,1954,1318,20957,13248
6,서울,중구,NaN,계,115665,9839,3896,7511,10304,9636,...,9313,9786,9002,6211,5077,4201,2581,1695,37325,19765
7,서울,중구,NaN,남자,55820,4988,1896,3103,5115,4790,...,4587,4830,4430,2998,2295,1795,1003,523,17998,8614


## Pivot_table 메소드로 필요한 열(인구수,20~39세, 65세 이상)만 가져오기
※ Pivot으로 못하는 일을 Pivot_table으로는 다 할 수 있으니, Pivot_table만 기억하면 됨

In [48]:
pop = pd.pivot_table(data   = pop, # 데이터프레임
                     index  = ['광역시도','시도'], # 행 위치에 들어갈 열
                     values = ['인구수', '20~39세', '65세 이상'], # 데이터로 사용할 열
                     columns= ['구분'], # 열 위치에 들어갈 열, values를 세분화하게 됨
                     aggfunc= 'sum' # 데이터 집계함수
                     )
pop.head()

20~39세               65세 이상                   인구수                
구분             계     남자     여자      계     남자     여자       계      남자      여자
광역시도 시도                                                                    
강원    강릉시  46008  25015  20993  40497  17378  23119  203057  101154  101903
      고성군   4006   2307   1699   7005   2957   4048   23621   11933   11688
      동해시  16311   8801   7510  16152   7075   9077   83888   42093   41795
      삼척시  12428   7148   5280  14953   6314   8639   61833   31630   30203
      속초시  15928   8517   7411  14010   5866   8144   76302   37488   38814

In [49]:
pop['소멸 비율'] = pop['20~39세','여자'] / pop['65세 이상','계'] 

In [50]:
pop['소멸 위기 지역'] = pop['소멸 비율'] < 0.5
pop.head()

20~39세               65세 이상                   인구수                  \
구분             계     남자     여자      계     남자     여자       계      남자      여자   
광역시도 시도                                                                       
강원    강릉시  46008  25015  20993  40497  17378  23119  203057  101154  101903   
      고성군   4006   2307   1699   7005   2957   4048   23621   11933   11688   
      동해시  16311   8801   7510  16152   7075   9077   83888   42093   41795   
      삼척시  12428   7148   5280  14953   6314   8639   61833   31630   30203   
      속초시  15928   8517   7411  14010   5866   8144   76302   37488   38814   

              소멸 비율 소멸 위기 지역  
구분                            
광역시도 시도                       
강원    강릉시  0.518384    False  
      고성군  0.242541     True  
      동해시  0.464958     True  
      삼척시  0.353106     True  
      속초시  0.528979    False

## Pivot_table 할 때 행 인덱스를 '광역시도', '시도' 멀티인덱스로 설정
- 때문에 '시도'만 보고 싶다면 꺼낼 때 get_level_values() 사용해야 함

In [51]:
pop[pop['소멸 위기 지역'] == True].head()

20~39세             65세 이상                인구수                \
구분             계    남자    여자      계    남자    여자      계     남자     여자   
광역시도 시도                                                                
강원    고성군   4006  2307  1699   7005  2957  4048  23621  11933  11688   
      동해시  16311  8801  7510  16152  7075  9077  83888  42093  41795   
      삼척시  12428  7148  5280  14953  6314  8639  61833  31630  30203   
      양구군   4656  2640  2016   4123  1784  2339  19900  10272   9628   
      양양군   4020  2210  1810   7405  3240  4165  25125  12599  12526   

              소멸 비율 소멸 위기 지역  
구분                            
광역시도 시도                       
강원    고성군  0.242541     True  
      동해시  0.464958     True  
      삼척시  0.353106     True  
      양구군  0.488964     True  
      양양군  0.244429     True

In [52]:
pop.index.get_level_values(1).to_numpy()[:10]

array([' 강릉시', ' 고성군', ' 동해시', ' 삼척시', ' 속초시', ' 양구군', ' 양양군', ' 영월군',
       ' 원주시', ' 인제군'], dtype=object)

## Pivot_table로 깔끔하게 정리는 됐지만 멀티컬럼즈가 되어버림
- 시각화를 위해 다시 단일컬럼즈로 초기화하는 작업

In [55]:
pop.columns.get_level_values(0), \
pop.columns.get_level_values(1)

(Index(['20~39세', '20~39세', '20~39세', '65세 이상', '65세 이상', '65세 이상', '인구수',
        '인구수', '인구수', '소멸 비율', '소멸 위기 지역'],
       dtype='object'),
 Index(['계', '남자', '여자', '계', '남자', '여자', '계', '남자', '여자', '', ''], dtype='object', name='구분'))

In [59]:
tmp_columns = [ pop.columns.get_level_values(0)[n] + pop.columns.get_level_values(1)[n] for n in range(len (pop.columns.get_level_values(0) ) )]
pop.columns = tmp_columns
pop.head()

20~39세계  20~39세남자  20~39세여자  65세 이상계  65세 이상남자  65세 이상여자    인구수계  \
광역시도 시도                                                                       
강원    강릉시    46008     25015     20993    40497     17378     23119  203057   
      고성군     4006      2307      1699     7005      2957      4048   23621   
      동해시    16311      8801      7510    16152      7075      9077   83888   
      삼척시    12428      7148      5280    14953      6314      8639   61833   
      속초시    15928      8517      7411    14010      5866      8144   76302   

            인구수남자   인구수여자     소멸 비율  소멸 위기 지역  
광역시도 시도                                        
강원    강릉시  101154  101903  0.518384     False  
      고성군   11933   11688  0.242541      True  
      동해시   42093   41795  0.464958      True  
      삼척시   31630   30203  0.353106      True  
      속초시   37488   38814  0.528979     False

In [61]:
pop.to_csv('./result/성별, 연령별 인구 - 시도(2019).csv', sep = ',', encoding = 'utf-8')